# Chapter 37: Configuration Basics (Practical, Not Mystical)

PostgreSQL's out-of-the-box configuration is designed for compatibility across hardware profiles, not performance. A stock installation typically allocates only 128MB of memory and conservative I/O settings suitable for a Raspberry Pi, not production workloads. This chapter provides a practical, operations-focused approach to tuning PostgreSQL without resorting to cargo-cult configuration copy-pasted from internet forums.

We focus on settings that yield measurable improvements while explaining the mechanics behind each parameter, enabling you to make informed decisions based on your specific hardware, workload, and consistency requirements.

## 37.1 Configuration Architecture and File Hierarchy

PostgreSQL reads configuration from several locations with specific precedence rules. Understanding this hierarchy is essential for maintaining configuration across environments and troubleshooting unexpected behavior.

### 37.1.1 Configuration File Locations

```sql
-- Locate configuration files dynamically
SELECT name, setting FROM pg_settings WHERE category = 'File Locations';
-- Typical output:
-- config_file          | /var/lib/postgresql/data/postgresql.conf
-- data_directory       | /var/lib/postgresql/data
-- external_pid_file    | 
-- hba_file             | /var/lib/postgresql/data/pg_hba.conf
-- ident_file           | /var/lib/postgresql/data/pg_ident.conf

-- Check if settings are from postgresql.conf or overridden elsewhere
SELECT name, setting, source, sourcefile, sourceline 
FROM pg_settings 
WHERE name IN ('shared_buffers', 'max_connections');
```

**File Hierarchy:**

1. **`postgresql.conf`**: Primary configuration file. Contains most server parameters.
2. **`postgresql.auto.conf`**: Machine-generated file managed via `ALTER SYSTEM` commands. Takes precedence over `postgresql.conf`.
3. **`pg_hba.conf`**: Host-Based Authentication configuration. Controls client authentication methods.
4. **`pg_ident.conf`**: User name mapping for external authentication systems.

**Configuration Precedence (highest to lowest):**
1. Command-line parameters (`postgres -c parameter=value`)
2. `postgresql.auto.conf` (ALTER SYSTEM)
3. `postgresql.conf`
4. Compiled-in defaults

### 37.1.2 Configuration Inclusion and Organization

Modern PostgreSQL deployments organize configuration using include directives rather than monolithic files.

```conf
-- postgresql.conf
-- Base settings
data_directory = '/var/lib/postgresql/16/main'
hba_file = '/etc/postgresql/16/main/pg_hba.conf'

-- Include directory for modular configuration
include_dir = 'conf.d'  -- Includes all .conf files in this directory alphabetically

-- Or include specific files
include_if_exists = 'custom_tuning.conf'
```

**Industry Standard Directory Structure:**

```bash
/etc/postgresql/16/main/
├── postgresql.conf          # Minimal base file with includes
├── pg_hba.conf             # Authentication rules
├── pg_ident.conf           # External user mapping
├── conf.d/
│   ├── 00-memory.conf      # Memory settings
│   ├── 10-storage.conf     # WAL, checkpoints, vacuum
│   ├── 20-replication.conf # Streaming replication settings
│   ├── 30-logging.conf     # Log configuration
│   └── 99-custom.conf      # Local overrides (gitignored)
└── environment             # System environment variables
```

**Benefits of this approach:**
- Version control friendly (separate files for different concerns)
- Environment-specific overlays (different memory settings for dev/prod)
- Ansible/Chef/Puppet can template individual files without parsing entire postgresql.conf
- Safe rollback (remove specific include file vs editing large config)

### 37.1.3 Applying Configuration Changes

PostgreSQL recognizes three classes of parameter changes:

```sql
-- Check context for any parameter (determines change method)
SELECT name, context, unit, short_desc 
FROM pg_settings 
WHERE name = 'shared_buffers';
-- context = 'postmaster'  → Requires restart
-- context = 'sighup'      → Reload configuration sufficient  
-- context = 'user'        → Can be changed per session
-- context = 'superuser'   → Can be changed by superuser for session/system
```

**Change Methods:**

1. **Restart Required** (`context = 'postmaster'`):
   ```bash
   # Stop and start (not reload)
   pg_ctl restart -D /var/lib/postgresql/data
   
   # Or systemd
   systemctl restart postgresql@16-main
   
   # Settings affected:
   # - shared_buffers, max_connections, listen_addresses
   # - port, unix_socket_directories, max_prepared_transactions
   ```

2. **Configuration Reload** (`context = 'sighup'`):
   ```bash   # Method 1: SQL command (superuser)
   SELECT pg_reload_conf();
   
   # Method 2: Signal to postmaster
   kill -HUP $(head -1 /var/lib/postgresql/data/postmaster.pid)
   
   # Method 3: pg_ctl
   pg_ctl reload -D /var/lib/postgresql/data
   
   # Method 4: systemd
   systemctl reload postgresql@16-main
   
   # Settings affected:
   # - autovacuum settings, log settings, ssl settings
   # - most connection/runtime tunables
   ```

3. **Per-Session Changes** (`context = 'user'` or `superuser'`):
   ```sql
   -- Current session only
   SET work_mem = '256MB';
   
   -- Current transaction only
   SET LOCAL work_mem = '1GB';
   
   -- For specific user (persistent)
   ALTER USER reporting_user SET work_mem = '512MB';
   
   -- For specific database
   ALTER DATABASE analytics SET work_mem = '1GB';
   ```

## 37.2 Memory Configuration

PostgreSQL memory architecture separates shared memory (accessible by all backends) from per-connection private memory. Misconfiguration here causes either OOM kills (Linux out-of-memory killer) or suboptimal performance.

### 37.2.1 Shared Buffers (The PostgreSQL Page Cache)

`shared_buffers` determines how much memory PostgreSQL dedicates to caching disk blocks. Unlike filesystem cache, PostgreSQL manages this directly with its own clock sweep algorithm.

```conf
# postgresql.conf
shared_buffers = 25GB  # Typically 25% of RAM on dedicated database servers
```

**Industry Guidelines:**

- **Dedicated PostgreSQL Server**: 25% of total RAM (not 50% as commonly mythologized)
  - Remaining memory available for:
    - Operating system filesystem cache (crucial for sequential scans)
    - Connection memory (work_mem × connections)
    - Maintenance operations
    - Application processes on same host (if any)

- **Shared Server** (PostgreSQL + Application): 15-20% of RAM

- **Containers/Cloud Instances**: Calculate based on cgroup limits, not host RAM:
  ```sql
  -- Check actual memory available to PostgreSQL process
  -- (Linux-specific, requires pg_read_file access)
  SELECT pg_read_file('/proc/meminfo');
  ```

**Why not 50% or more?**
PostgreSQL relies heavily on the operating system's filesystem cache for:
- Double buffering avoidance (kernel cache + shared_buffers)
- Efficient sequential scan performance
- WAL file caching
- Temporary file handling

```sql
-- Verify shared_buffers effectiveness (cache hit ratio should be >99%)
SELECT 
    sum(heap_blks_read) as heap_read,
    sum(heap_blks_hit)  as heap_hit,
    sum(heap_blks_hit) / (sum(heap_blks_hit) + sum(heap_blks_read)) * 100 as ratio
FROM pg_statio_user_tables;
-- If ratio < 95%, increase shared_buffers (if memory available)
-- If ratio > 99.5% and memory pressure exists, can reduce slightly
```

**Huge Pages (Linux Optimization):**

For servers with >32GB RAM and large shared_buffers, enable Huge Pages to reduce TLB (Translation Lookaside Buffer) misses:

```bash
# /etc/sysctl.conf
vm.nr_hugepages = 13312  # Calculation: shared_buffers / huge_page_size (usually 2MB)
# 25GB / 2MB = ~12800, add 5% buffer = 13312

# postgresql.conf
huge_pages = try  # 'on' = fail if unavailable, 'try' = use if available, 'off' = never
```

### 37.2.2 Work Memory (Query Operations)

`work_mem` specifies the amount of memory available for internal sort operations and hash tables before spilling to disk. Unlike shared_buffers, work_mem is allocated **per operation**, not per query or connection.

```conf
# Base setting (conservative for OLTP)
work_mem = 4MB

# Analytics/reporting database
work_mem = 256MB
```

**Critical Understanding:**
If a query performs:
- 2 sorts (ORDER BY + merge join)
- 1 hash join
- 3 hash aggregations

It can consume up to `6 × work_mem` per connection.

**Calculation Formula:**

```sql
-- Safe work_mem calculation
-- Available RAM = Total RAM - shared_buffers - (OS + other processes)
-- Conservative: Available RAM / max_connections / 2 (average operations per query)
-- Aggressive: Available RAM / max_connections / 4

-- Example: 64GB RAM, shared_buffers=16GB, max_connections=200
-- Available: 48GB
-- Conservative: 48GB / 200 / 2 = 120MB
-- Safe setting: work_mem = 64MB to 128MB depending on workload mix
```

**Monitoring Disk Spills:**

```sql
-- Identify queries spilling to disk (temp files)
SELECT 
    pid,
    usename,
    query,
    temp_blks_read + temp_blks_written as temp_blocks,
    pg_size_pretty((temp_blks_read + temp_blks_written) * 8192) as temp_size
FROM pg_stat_activity
JOIN pg_stat_user_tables USING (relid)
WHERE temp_blks_read + temp_blks_written > 0
ORDER BY temp_blocks DESC;

-- Historical analysis (requires pg_stat_statements)
SELECT 
    query,
    calls,
    mean_exec_time,
    temp_blks_written * 8 / 1024 as temp_mb_written
FROM pg_stat_statements
WHERE temp_blks_written > 0
ORDER BY temp_blks_written DESC
LIMIT 10;
```

**Per-Query Override Pattern:**

```sql
-- Increase work_mem for specific heavy reporting query
BEGIN;
SET LOCAL work_mem = '1GB';
SELECT * FROM huge_table ORDER BY complex_expression;
COMMIT;  -- work_mem reverts to default after commit

-- Or set for specific user role
ALTER ROLE analytics_app SET work_mem = '512MB';
ALTER ROLE web_app SET work_mem = '8MB';
```

### 37.2.3 Maintenance Work Memory

`maintenance_work_mem` controls memory for maintenance operations: VACUUM, CREATE INDEX, ALTER TABLE ADD FOREIGN KEY, and REINDEX.

```conf
maintenance_work_mem = 1GB  # Default 64MB is too low for production tables
```

**Guidelines:**
- **VACUUM**: Can use up to 1GB effectively per worker (autovacuum_max_workers)
- **CREATE INDEX**: More memory = faster sorts, especially for B-tree creation
- **Adding FKs**: Requires table scan and validation; memory speeds this significantly

```sql
-- Temporarily increase for specific maintenance
SET maintenance_work_mem = '8GB';
CREATE INDEX CONCURRENTLY idx_large_table ON large_table (column1, column2);
RESET maintenance_work_mem;
```

**Autovacuum-Specific Memory:**

```conf
autovacuum_work_mem = -1  # -1 means use maintenance_work_mem
# Or set independently:
autovacuum_work_mem = 512MB
```

### 37.2.4 Effective Cache Size (Planner Hint)

`effective_cache_size` informs the query planner about the total available cache (shared_buffers + OS filesystem cache). It does **not** allocate memory; it only influences cost estimates.

```conf
effective_cache_size = 48GB  # Total RAM - shared_buffers - headroom
```

**Best Practice:**
Set to approximately `(Total RAM - shared_buffers) × 0.9`. On a 64GB server with 16GB shared_buffers: `(48GB × 0.9) = ~43GB`.

**Why it matters:**
The planner chooses between index scans and sequential scans based on estimated cache hit probability. Underestimating causes unnecessary index usage; overestimating causes slow sequential scans.

```sql
-- Verify planner assumptions match reality
EXPLAIN (ANALYZE, BUFFERS) SELECT * FROM large_table WHERE id > 1000;
-- Check if "Shared Hit Blocks" matches planner expectations
```

### 37.2.5 Connection Memory Overhead

Each connection consumes memory regardless of activity:

```conf
# Memory per connection (approximate)
# - Base overhead: ~1.5MB
# - work_mem (worst case if multiple operations)
# - temp_buffers: 8MB default (session-local temporary tables)
# - max_prepared_transactions overhead if using prepared xacts

# Calculate maximum theoretical memory usage:
# shared_buffers + (max_connections × (work_mem × avg_ops_per_query + 1.5MB))
```

**Connection Pooling Strategy:**
Since memory scales with connections, always use connection pooling (PgBouncer) in production:

```conf
# With PgBouncer transaction pooling:
max_connections = 200          # Database level
pgbouncer default_pool_size = 20  # Actual Postgres connections
application connections = 1000    # Can be much higher
```

## 37.3 Write-Ahead Logging (WAL) Configuration

The WAL is the center of PostgreSQL's durability and replication architecture. Proper configuration balances crash recovery time (RTO) against write performance.

### 37.3.1 WAL Level and Archiving

```conf
# wal_level determines what information is written to WAL
wal_level = replica  # Options: minimal, replica, logical

# minimal: Only crash recovery (cannot replicate or use PITR)
# replica: Supports streaming replication and PITR (most common)
# logical: Supports logical replication and logical decoding
```

**Industry Standard:**
- **Standalone/Development**: `minimal` (if you don't need replication)
- **Production**: `replica` (minimum for backups and HA)
- **Logical Replication**: `logical` (required for pglogical, pgoutput)

### 37.3.2 Checkpoint Configuration

Checkpoints ensure dirty buffers are written to disk and WAL can be recycled. Aggressive checkpointing improves crash recovery time but hurts performance; relaxed checkpointing improves performance but extends recovery time.

```conf
# Time-based checkpointing
checkpoint_timeout = 15min        # Range: 30s to 1d, default 5min
checkpoint_completion_target = 0.9 # Spread writes over 90% of checkpoint interval

# Size-based checkpointing (Primary control in modern Postgres)
max_wal_size = 4GB               # Default 1GB too small for busy systems
min_wal_size = 1GB               # WAL recycling target
```

**Mechanism Explanation:**
When WAL reaches `max_wal_size`, PostgreSQL initiates a checkpoint. With `checkpoint_completion_target = 0.9`, PostgreSQL spreads disk writes over `15min × 0.9 = 13.5 minutes` to avoid I/O spikes.

**Tuning for SSD vs HDD:**

```conf
# SSD/NVMe (fast random I/O):
checkpoint_timeout = 15min
max_wal_size = 8GB to 32GB  # Larger = fewer checkpoints, but longer recovery
checkpoint_completion_target = 0.9

# Traditional HDD (slow seeks, benefit from fewer writes):
checkpoint_timeout = 10min
max_wal_size = 4GB
checkpoint_completion_target = 0.5  # Complete faster to free buffers
```

**Monitoring Checkpoint Behavior:**

```sql
-- Check checkpoint frequency and timing
SELECT 
    checkpoints_timed,      -- Scheduled by timeout
    checkpoints_req,        -- Forced by WAL size (should be minority)
    checkpoint_write_time,  -- Time writing buffers to disk (milliseconds)
    checkpoint_sync_time    -- Time syncing to disk (fsync)
FROM pg_stat_bgwriter;

-- Ideal ratio: checkpoints_timed should be 80%+ of total checkpoints
-- If checkpoints_req dominates, increase max_wal_size
```

### 37.3.3 WAL Buffers

```conf
wal_buffers = -1  # Auto-tuned: typically 1/32 of shared_buffers, capped at 16MB
```

**Manual Override:**
Only change from `-1` if you observe WAL buffer waits:

```sql
-- Check for WAL write waits
SELECT 
    wal_buffer_writes,  -- Times WAL data written to disk
    wal_buffers_full    -- Times WAL buffers were full and waited
FROM pg_stat_wal;  -- PostgreSQL 14+
```

**Guideline:**
For high-write systems ( >1000 TPS), explicitly set:
```conf
wal_buffers = 16MB  # Maximum effective size
```

### 37.3.4 Synchronous Commit Modes

`synchronous_commit` balances durability guarantees against commit latency.

```conf
synchronous_commit = on  # Default: wait for WAL flush to disk before acknowledging commit
```

**Options:**

1. **`on`** (Production Default):
   - Waits for WAL to reach disk before returning success to client
   - Guarantees durability even if OS crashes
   - Latency: Disk fsync time (0.5-5ms SSD, 5-20ms HDD)

2. **`off`** (High-throughput, acceptable data loss window):
   - Returns success immediately; WAL flushed by wal_writer process
   - Risk: Last few seconds of transactions lost on crash
   - Use case: Logging, analytics bulk loads, non-critical writes
   
   ```sql
   -- Per-session for bulk load
   BEGIN;
   SET LOCAL synchronous_commit = off;
   -- ... bulk insert ...
   COMMIT;
   ```

3. **`local`**:
   - Waits for local disk flush, not replica confirmation
   - Used when streaming replication has `synchronous_standby_names` set but you don't want to wait for network round-trip

4. **`remote_write`**, **`remote_apply`**, **`on`** (with sync replication):
   - Wait for standby to receive/write/apply WAL
   - See Chapter 33 for streaming replication details

## 37.4 Autovacuum Tuning

Autovacuum prevents transaction ID wraparound and maintains query performance by updating table statistics and reclaiming dead tuples. Default settings are too conservative for high-churn tables.

### 37.4.1 Global Autovacuum Settings

```conf
autovacuum = on                    # Master switch
autovacuum_max_workers = 3         # Parallel vacuum processes (default 3)
autovacuum_naptime = 1min          # Check interval
autovacuum_vacuum_threshold = 50   # Minimum dead tuples before vacuum
autovacuum_vacuum_scale_factor = 0.2  # Fraction of table size to trigger vacuum
autovacuum_analyze_threshold = 50
autovacuum_analyze_scale_factor = 0.1
```

**Worker Memory and I/O:**

```conf
autovacuum_work_mem = -1           # Use maintenance_work_mem
autovacuum_vacuum_cost_delay = 2ms  # Throttling (default 2ms)
autovacuum_vacuum_cost_limit = -1   # -1 = use vacuum_cost_limit (default 10000)
```

**Cost-Based Throttling Explanation:**
PostgreSQL assigns "cost points" to vacuum operations:
- 1 for each page hit (buffer pool)
- 10 for each page miss (disk read)
- 20 for each dirty page written

When accumulated cost reaches `autovacuum_vacuum_cost_limit`, vacuum sleeps for `autovacuum_vacuum_cost_delay`.

**Modern SSD Tuning:**
For SSD storage, increase the cost limit to allow faster vacuuming:

```conf
autovacuum_vacuum_cost_limit = 2000   # Double the default
autovacuum_vacuum_cost_delay = 2ms    # Or even 0ms for very fast storage
```

### 37.4.2 Per-Table Autovacuum Configuration

High-churn tables need aggressive vacuuming; large static tables need minimal attention.

```sql
-- Aggressive settings for high-update table (e.g., session store)
ALTER TABLE user_sessions SET (
    autovacuum_vacuum_scale_factor = 0.05,  -- Vacuum at 5% dead tuples (vs 20%)
    autovacuum_analyze_scale_factor = 0.02, -- Analyze at 2% changes
    autovacuum_vacuum_cost_limit = 1000,    -- Faster vacuum (less throttling)
    fillfactor = 85                         -- Leave 15% free for HOT updates
);

-- Minimal vacuuming for append-only large table (e.g., audit log)
ALTER TABLE audit_log SET (
    autovacuum_vacuum_scale_factor = 0.4,   -- Only vacuum at 40% dead tuples
    autovacuum_analyze_scale_factor = 0.0,  -- Disable auto-analyze
    autovacuum_analyze_threshold = 0
);
-- Manually analyze during low-traffic windows instead
```

**Formula for Scale Factor:**
Default `scale_factor = 0.2` on a 100GB table requires 20GB of dead tuples before vacuuming. Use `autovacuum_vacuum_insert_scale_factor` (PG13+) or absolute thresholds instead:

```sql
-- PostgreSQL 13+: Vacuum based on insert count (for anti-wraparound)
ALTER TABLE high_insert_table SET (
    autovacuum_vacuum_insert_threshold = 1000,
    autovacuum_vacuum_insert_scale_factor = 0.05
);

-- Absolute threshold (better for large tables)
ALTER TABLE large_table SET (
    autovacuum_vacuum_threshold = 10000,
    autovacuum_vacuum_scale_factor = 0.0  -- Ignore scale, use absolute threshold
);
-- Now vacuums when 10,000 dead tuples exist, regardless of table size
```

### 37.4.3 Freeze Thresholds and Anti-Wraparound

Transaction ID wraparound is a critical failure mode. PostgreSQL forces vacuum when tables approach the 2 billion transaction limit.

```conf
vacuum_freeze_min_age = 50000000        # 50 million transactions
vacuum_freeze_table_age = 150000000     # 150 million (forces aggressive vacuum)
autovacuum_freeze_max_age = 200000000   # 200 million (forces autovacuum regardless of load)
```

**Monitoring Wraparound Risk:**

```sql
-- Critical: Check age of databases (must stay below 2 billion)
SELECT 
    datname,
    age(datfrozenxid) as xid_age,
    2000000000 - age(datfrozenxid) as transactions_until_wraparound,
    pg_size_pretty(pg_database_size(oid)) as db_size
FROM pg_database
ORDER BY age(datfrozenxid) DESC;

-- Table-level detail
SELECT 
    relname,
    age(relfrozenxid) as xid_age,
    n_dead_tup,
    last_vacuum,
    last_autovacuum
FROM pg_stat_user_tables
ORDER BY age(relfrozenxid) DESC
LIMIT 20;
```

**Emergency Intervention:**
If `age(relfrozenxid)` approaches 1.5 billion:
```sql
-- Vacuum with freeze (aggressive, blocks writes briefly)
VACUUM (FREEZE, ANALYZE, VERBOSE) critical_table;
```

## 37.5 Environment-Specific Configuration Strategy

Configuration must adapt to development, staging, and production constraints while maintaining consistency in logic (e.g., SQL behavior settings).

### 37.5.1 Development Environment

```conf
# 00-memory-dev.conf
shared_buffers = 256MB          # Small, laptop-friendly
work_mem = 16MB                 # Allow complex queries during development
maintenance_work_mem = 256MB    # Fast index creation on small datasets

# Logging everything for debugging
log_statement = 'all'           # Log every SQL statement
log_duration = on
log_min_duration_statement = 0  # Log all durations
log_line_prefix = '%t [%p]: [%l-1] user=%u,db=%d,app=%a,client=%h '
log_checkpoints = on
log_connections = on
log_disconnections = on

# Relaxed durability for speed
synchronous_commit = off        # Faster, acceptable for dev data loss
fsync = off                     # NEVER in production, acceptable for local dev
full_page_writes = off          # NEVER in production
```

**Critical Safety:** Always use `ALTER SYSTEM` or distinct include files to prevent dev settings reaching production:

```bash
# Production deployment checklist
grep -E "^(fsync|synchronous_commit|full_page_writes)" postgresql.conf
# Should never see fsync = off in production
```

### 37.5.2 Staging/Testing Environment

Mirror production hardware at smaller scale but maintain identical configuration logic:

```conf
# Staging should test production parameters
shared_buffers = 2GB            # Scaled down from prod 32GB
work_mem = 4MB                  # Same as production
maintenance_work_mem = 512MB    # Same ratio as production

# But enable heavy logging for performance testing
log_min_duration_statement = 100  # Log slow queries for optimization
auto_explain.log_min_duration = 100
auto_explain.log_analyze = on     # See actual plans in logs
```

### 37.5.3 Production Environment

```conf
# 00-memory-prod.conf
shared_buffers = 32GB           # 25% of 128GB RAM
work_mem = 8MB                  # Conservative base, override per-role if needed
maintenance_work_mem = 2GB
effective_cache_size = 96GB     # (128GB - 32GB) * 0.9

# 10-storage-prod.conf
max_wal_size = 16GB
checkpoint_timeout = 15min
checkpoint_completion_target = 0.9
wal_buffers = 16MB

# 20-vacuum-prod.conf
autovacuum_max_workers = 6      # More workers for high-churn OLTP
autovacuum_vacuum_cost_limit = 2000  # Faster vacuum on SSD storage

# 30-logging-prod.conf (minimal, for performance)
log_min_duration_statement = 1000   # Only log queries > 1s
log_line_prefix = '%t [%p]: [%l-1] user=%u,db=%d,app=%a,client=%h '
log_checkpoints = on                # Always monitor checkpoints
log_lock_waits = on                 # Detect deadlock/lock timeout
log_temp_files = 32MB               # Log disk sorts > 32MB
```

### 37.5.4 Configuration Validation Checklist

Before applying configuration changes:

```sql
-- 1. Verify no typos (invalid parameter names)
SELECT name FROM pg_settings WHERE source = 'configuration file';

-- 2. Check for pending restarts (parameters changed but require restart)
SELECT name, setting, pending_restart 
FROM pg_settings 
WHERE pending_restart = true;

-- 3. Validate memory math won't cause OOM
WITH mem_settings AS (
    SELECT 
        (SELECT setting::int * 8192 FROM pg_settings WHERE name = 'shared_buffers') as shared_bytes,
        (SELECT setting::int FROM pg_settings WHERE name = 'max_connections') as max_conn,
        (SELECT setting::int * 1024 FROM pg_settings WHERE name = 'work_mem') as work_bytes,
        (SELECT setting::int * 1024 FROM pg_settings WHERE name = 'maintenance_work_mem') as maint_bytes,
        (SELECT setting::int FROM pg_settings WHERE name = 'autovacuum_max_workers') as vac_workers
)
SELECT 
    pg_size_pretty(shared_bytes) as shared_buffers,
    max_conn,
    pg_size_pretty(work_bytes) as work_mem,
    -- Worst case memory (all connections doing 2 sorts + autovacuum maintenance)
    pg_size_pretty(
        shared_bytes + 
        (max_conn * work_bytes * 2) + 
        (vac_workers * maint_bytes)
    ) as theoretical_max_memory
FROM mem_settings;
```

## 37.6 Critical Parameters Reference

### 37.6.1 Connection and Security

```conf
max_connections = 200           # Increase only if necessary; prefer pooling
superuser_reserved_connections = 3  # Reserved for emergency access
ssl = on
ssl_cert_file = 'server.crt'
ssl_key_file = 'server.key'
password_encryption = scram-sha-256  # Modern standard, not md5
```

### 37.6.2 Query Planning

```conf
random_page_cost = 1.1          # For SSD storage (default 4.0 is for HDD)
seq_page_cost = 1.0
effective_io_concurrency = 200  # SSD random read capability
```

**Why `random_page_cost` matters:**
Default `4.0` assumes random I/O is 4x slower than sequential. On SSDs, random and sequential performance are nearly equal (`1.1` or `1.0`). Setting this incorrectly causes the planner to avoid index scans in favor of sequential scans.

### 37.6.3 Logging and Monitoring

```conf
logging_collector = on
log_directory = 'log'
log_filename = 'postgresql-%Y-%m-%d_%H%M%S.log'
log_rotation_age = 1d
log_rotation_size = 100MB

# Performance insights
track_io_timing = on            # Enable pg_stat_database blk_read_time
track_functions = pl            # Track function execution times
```

## Chapter Summary

In this chapter, you learned:

1. **Configuration Architecture**: PostgreSQL uses `postgresql.conf` (base), `postgresql.auto.conf` (ALTER SYSTEM overrides), and include directories for modular management. Always organize production configs into logical include files (memory.conf, storage.conf, replication.conf).

2. **Memory Management**: 
   - `shared_buffers`: 25% of RAM on dedicated servers, never exceed 40%
   - `work_mem`: Conservative base (4-8MB), override per-session for analytics
   - `maintenance_work_mem`: 1-2GB for fast index creation and vacuuming
   - `effective_cache_size`: Planner hint set to ~70% of total RAM

3. **WAL and Checkpoints**: 
   - `wal_level = replica` minimum for production (enables PITR and replication)
   - `max_wal_size`: 4GB-32GB depending on write volume and recovery time objectives
   - `checkpoint_completion_target = 0.9` to spread I/O evenly
   - `synchronous_commit`: `on` for durability, `off` only for specific high-throughput batch loads

4. **Autovacuum Tuning**: 
   - Increase `autovacuum_max_workers` to 4-6 for high-churn OLTP systems
   - Reduce `autovacuum_vacuum_scale_factor` (to 0.05 or 0.1) for large tables using `ALTER TABLE`
   - Monitor `pg_stat_user_tables` to ensure vacuum keeps up with dead tuple generation
   - Watch transaction ID age (`age(relfrozenxid)`) to prevent wraparound emergencies

5. **Environment Strategy**: 
   - Development: Relaxed durability (`fsync = off` acceptable only locally), verbose logging
   - Production: Conservative memory settings, minimal logging (slow queries only), maximum durability
   - Always validate configuration with `pg_settings` views before and after changes

6. **Change Management**: Distinguish between reloadable parameters (SIGHUP) and restart-required parameters (postmaster). Use `SELECT pg_reload_conf()` for online tuning where possible.

---

**Next:** In Chapter 38, we will examine Vacuum, Analyze, and Bloat management in detail—covering the mechanics of MVCC cleanup, table bloat detection and remediation, aggressive vacuuming strategies for high-churn tables, and the pg_dump/pg_repack tools for physical reorganization.